<a href="https://colab.research.google.com/github/aarav2703/mnist-digit-classification/blob/main/dig_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img


In [10]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize the data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Check the shape of the data
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


x_train shape: (60000, 28, 28, 1)
y_train shape: (60000, 10)
x_test shape: (10000, 28, 28, 1)
y_test shape: (10000, 10)


In [11]:
def build_cnn():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn()
cnn_model.summary()

# Train the CNN model
cnn_model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Evaluate the CNN model
cnn_loss, cnn_accuracy = cnn_model.evaluate(x_test, y_test)
print(f"CNN Accuracy: {cnn_accuracy:.4f}")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 5, 5, 64)         

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)

# Train the CNN model with augmented data
cnn_model.fit(datagen.flow(x_train, y_train, batch_size=128), epochs=10, validation_data=(x_test, y_test))

# Evaluate the CNN model after augmentation
cnn_loss, cnn_accuracy = cnn_model.evaluate(x_test, y_test)
print(f"CNN Accuracy after Augmentation: {cnn_accuracy:.4f}")


Epoch 1/10
469/469 [==============================] - 18s 37ms/step - loss: 0.2017 - accuracy: 0.9403 - val_loss: 0.0221 - val_accuracy: 0.9912
Epoch 2/10
469/469 [==============================] - 20s 42ms/step - loss: 0.1358 - accuracy: 0.9587 - val_loss: 0.0198 - val_accuracy: 0.9926
Epoch 3/10
469/469 [==============================] - 18s 38ms/step - loss: 0.1215 - accuracy: 0.9636 - val_loss: 0.0240 - val_accuracy: 0.9915
Epoch 4/10
469/469 [==============================] - 18s 39ms/step - loss: 0.1103 - accuracy: 0.9671 - val_loss: 0.0172 - val_accuracy: 0.9941
Epoch 5/10
469/469 [==============================] - 18s 38ms/step - loss: 0.1012 - accuracy: 0.9695 - val_loss: 0.0157 - val_accuracy: 0.9946
Epoch 6/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0979 - accuracy: 0.9706 - val_loss: 0.0175 - val_accuracy: 0.9938
Epoch 7/10
469/469 [==============================] - 18s 38ms/step - loss: 0.0917 - accuracy: 0.9731 - val_loss: 0.0171 - val_accuracy:

In [13]:
# Flatten the data for KNN
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# KNN model
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train_flat, np.argmax(y_train, axis=1))
knn_predictions = knn.predict(x_test_flat)

# Evaluate KNN model
knn_accuracy = accuracy_score(np.argmax(y_test, axis=1), knn_predictions)
print(f"KNN Accuracy: {knn_accuracy:.4f}")


KNN Accuracy: 0.9705


In [6]:
# PSVM model with probability enabled
psvm = SVC(kernel='poly', degree=3, probability=True)
psvm.fit(x_train_flat, np.argmax(y_train, axis=1))
psvm_predictions = psvm.predict_proba(x_test_flat)

# Evaluate PSVM model
psvm_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(psvm_predictions, axis=1))
print(f"PSVM Accuracy: {psvm_accuracy:.4f}")


PSVM Accuracy: 0.9788


In [14]:
def build_nn():
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

nn_model = build_nn()
nn_model.summary()

# Train the Neural Network
nn_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate Neural Network model
nn_loss, nn_accuracy = nn_model.evaluate(x_test, y_test)
print(f"Neural Network Accuracy: {nn_accuracy:.4f}")


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               100480    
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 10)                650       
                                                                 
Total params: 109386 (427.29 KB)
Trainable params: 109386 (427.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
187

In [15]:
# Resize function
def resize_images(images, new_size):
    resized_images = np.zeros((images.shape[0], new_size[0], new_size[1], 3))
    for i in range(images.shape[0]):
        img = array_to_img(images[i])
        img = img.resize(new_size)
        resized_images[i] = img_to_array(img)
    return resized_images

# Resize MNIST images to 32x32 and repeat the grayscale channel to fit VGG16 input
x_train_resized = resize_images(np.repeat(x_train, 3, axis=-1), (32, 32))
x_test_resized = resize_images(np.repeat(x_test, 3, axis=-1), (32, 32))

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the final model
transfer_model = Model(inputs=base_model.input, outputs=predictions)
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the transfer learning model
transfer_model.fit(x_train_resized, y_train, epochs=10, validation_data=(x_test_resized, y_test))

# Evaluate the transfer learning model
transfer_loss, transfer_accuracy = transfer_model.evaluate(x_test_resized, y_test)
print(f"Transfer Learning (VGG16) Accuracy: {transfer_accuracy:.4f}")


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.8619 - accuracy: 0.7943 - val_loss: 0.2453 - val_accuracy: 0.9171
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3673 - accuracy: 0.8834 - val_loss: 0.2018 - val_accuracy: 0.9362
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3214 - accuracy: 0.8982 - val_loss: 0.1868 - val_accuracy: 0.9407
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2993 - accuracy: 0.9050 - val_loss: 0.1798 - val_accuracy: 0.9439
Epoch 5/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2747 - accuracy: 0.9125 - val_loss: 0.1817 - val_accuracy: 0.9430
Epoch 6/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2647 - accuracy: 0.9154 - val_loss: 0.1696 - val_accuracy: 0.9464
Epoch 7/10
1875/1875 [==============================] - 18s 10ms

In [16]:
# Define function to get model predictions
def get_predictions(model, data, model_type='keras'):
    if model_type == 'keras':
        return model.predict(data)
    else:
        return model.predict_proba(data)

# Predictions from individual models
cnn_predictions = get_predictions(cnn_model, x_test)
nn_predictions = get_predictions(nn_model, x_test)
knn_predictions = knn.predict_proba(x_test_flat)
psvm_predictions = psvm.predict_proba(x_test_flat)

# Convert Keras model predictions to match the shape of scikit-learn predictions
cnn_predictions = cnn_predictions.reshape(-1, 10)
nn_predictions = nn_predictions.reshape(-1, 10)

# Averaging the predictions (soft voting)
ensemble_predictions = (cnn_predictions + nn_predictions + knn_predictions + psvm_predictions) / 4.0
ensemble_predictions = np.argmax(ensemble_predictions, axis=1)

# Evaluate ensemble model
ensemble_accuracy = accuracy_score(np.argmax(y_test, axis=1), ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


313/313 [==============================] - 1s 2ms/step
Ensemble Accuracy: 0.9892


In [47]:
# TODO: Add code for modern architectures
from tensorflow.keras.applications import ResNet50, InceptionV3

# Load ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# Freeze layers
for layer in resnet_model.layers:
    layer.trainable = False

# Add custom layers
x = resnet_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create model
resnet_model_final = Model(inputs=resnet_model.input, outputs=predictions)
resnet_model_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet_model_final.fit(x_train_resized, y_train, epochs=10, validation_data=(x_test_resized, y_test))

# Evaluate the model
resnet_loss, resnet_accuracy = resnet_model_final.evaluate(x_test_resized, y_test)
print(f"ResNet50 Accuracy: {resnet_accuracy:.4f}")


94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
1875/1875 [==============================] - 27s 12ms/step - loss: 0.5817 - accuracy: 0.8120 - val_loss: 0.2121 - val_accuracy: 0.9298
Epoch 2/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3718 - accuracy: 0.8791 - val_loss: 0.1737 - val_accuracy: 0.9448
Epoch 3/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.3262 - accuracy: 0.8957 - val_loss: 0.1919 - val_accuracy: 0.9402
Epoch 4/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.3086 - accuracy: 0.9002 - val_loss: 0.1619 - val_accuracy: 0.9477
Epoch 5/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.2882 - accuracy: 0.9072 - val_loss: 0.1620 - val_accuracy: 0.9499
Epoch 6/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.2758 - accuracy: 0.9107 - val_loss: 0.1459 - val_accuracy: 0.9540
Epoch 7/10
1875/1875 [==============================] - 25s

In [48]:
# TODO: Add code for performance analysis
import time

def measure_performance(model, data, labels):
    start_time = time.time()
    loss, accuracy = model.evaluate(data, labels, verbose=0)
    end_time = time.time()
    return accuracy, end_time - start_time

cnn_accuracy, cnn_time = measure_performance(cnn_model, x_test, y_test)
nn_accuracy, nn_time = measure_performance(nn_model, x_test, y_test)
ensemble_accuracy, ensemble_time = measure_performance(nn_model, x_test, y_test)  # Using nn_model as a placeholder

print(f"CNN Accuracy: {cnn_accuracy:.4f}, Time: {cnn_time:.4f}s")
print(f"NN Accuracy: {nn_accuracy:.4f}, Time: {nn_time:.4f}s")
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}, Time: {ensemble_time:.4f}s")


CNN Accuracy: 0.9941, Time: 1.4522s
NN Accuracy: 0.9774, Time: 1.1251s
Ensemble Accuracy: 0.9774, Time: 0.7169s


In [49]:
# TODO: Add code for model deployment
import tensorflow as tf

# Save the model
cnn_model.save('cnn_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('cnn_model.h5')

# Verify the loaded model
loss, accuracy = loaded_model.evaluate(x_test, y_test)
print(f"Loaded model accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 1s 4ms/step - loss: 0.0165 - accuracy: 0.9941
Loaded model accuracy: 0.9941


In [50]:
# TODO: Add code for edge computing
import tensorflow as tf
import tensorflow.lite as tflite

# Convert the model to TensorFlow Lite format
converter = tflite.TFLiteConverter.from_keras_model(cnn_model)
tflite_model = converter.convert()

# Save the model
with open('cnn_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Load and run the model on a TFLite interpreter
interpreter = tflite.Interpreter(model_path='cnn_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_data = np.array(np.random.random_sample(input_details[0]['shape']), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# Get the results
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7f64902a2170>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7f64902a2170>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[[5.1477941e-06 1.0067246e-07 1.5272357e-04 1.1641455e-04 1.2558603e-06
  4.6983416e-05 4.5607601e-05 1.3279307e-08 9.9956852e-01 6.3257226e-05]]


In [1]:
pip freeze > requirements.txt
